In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import hashlib
import pandas as pd
import time

In [2]:
# Given an html hyperlink, will check for consecutive digits 
# in the string defined by 'ID' and return the set it found
def order_id(ahref):
    ids = []
    for link in ahref:
        if 'ID' in link['href']:
            web_id = re.search('[0-9]+', link['href']).group(0)
            ids.append(int(web_id))

    ids.sort()
    return ids

# base url
url = "http://www.gebaeudebrueter-in-berlin.de/index.php"

# have the database return a html page with the IDs of all the records (% is wildcard)
content = urlopen(url + "?find=%25&x=0&y=0").read()
soup = BeautifulSoup(content, features="html.parser")

# have Soup select all the hyperlinks and order them
ahref = soup.findAll('a', {'href': True})

# strip everything, sort and store the numerical IDs
ordered_ids = order_id(ahref)


In [4]:
def get_data(web_id):
    detailContent = urlopen(url + '?ID=' + str(web_id)).read()
    soup = BeautifulSoup(detailContent, features="html.parser")

    # table returns a bs4.element.ResultSet
    table = soup.findAll('table')

    # the checkboxes are housed in the 5th table in the html file
    # only top-level <table> tags are counted not their nested children
    checkboxes = table[4]

    # in the 5th table find all children of the tag <td>
    td = checkboxes.findChildren('td')

    # in the n-th record of the ResultSet of <td> find the children (i.e. the value) 
    # of the attribute tag 'input
    bezirk = td[1].findChildren('input')[0]['value']
    mauersegler = 1 if td[2].findChildren('input', checked=True) else 0
    kontrolle = 1 if td[3].findChildren('input', checked=True) else 0

    plz = td[5].findChildren('input')[0]['value']
    sperling = 1 if td[6].findChildren('input', checked=True) else 0
    ersatz = 1 if td[7].findChildren('input', checked=True) else 0

    ort = td[9].findChildren('input')[0]['value']
    schwalbe = 1 if td[10].findChildren('input', checked=True) else 0
    wichtig = 1 if td[11].findChildren('input', checked=True) else 0

    strasse = td[13].findChildren('input')[0]['value']
    star = 1 if td[14].findChildren('input', checked=True) else 0
    sanierung = 1 if td[15].findChildren('input', checked=True) else 0

    anhang = td[17].findChildren('input')[0]['value']
    fledermaus = 1 if td[18].findChildren('input', checked=True) else 0
    verloren = 1 if td[19].findChildren('input', checked=True) else 0

    erstbeobachtung = td[21].findChildren('input')[0]['value']
    andere = 1 if td[22].findChildren('input', checked=True) else 0
    #melder = td[24].findChildren('input')[0]['value']
    
    # text fields are in the 6th html table
    text_fields = table[5]
    td = text_fields.findChildren('td')
    beschreibung = td[1].findChildren('textarea')[0].text
    besonderes = td[3].findChildren('textarea')[0].text
    #internes = td[5].findChildren('textarea')[0].text

    # pack all the collected data into a list
    data = (web_id, bezirk, plz, ort, strasse, anhang, erstbeobachtung, beschreibung, besonderes, mauersegler,
            kontrolle, sperling, ersatz, schwalbe, wichtig, star, sanierung, fledermaus, verloren, andere)

    # pack all the collected data and calculate a hash for it
    # this can be used later to check if a database record has been upated
    payload = ''.join(map(str, data))
    checksum = hashlib.sha3_224(payload.encode('utf-8')).hexdigest()

    return data + (checksum,) # pretend checksum is a tuple by adding a comma


list_brueter = []

# just for checking
total = len(ordered_ids)

for web_id in ordered_ids:
    # print where you are scraper so that I know you are working
    print("ID", web_id, "of", total)
    
    # be polite, limit your requests to one per 10 seconds
    time.sleep(10)

    values = get_data(web_id)

    list_brueter.append(values)


ID 1 of 2254
ID 2 of 2254
ID 3 of 2254
ID 4 of 2254
ID 5 of 2254
ID 6 of 2254
ID 7 of 2254
ID 8 of 2254
ID 10 of 2254
ID 11 of 2254
ID 12 of 2254


KeyboardInterrupt: 

Scraping took 385m 9.2s

In [4]:
# pack the list of scaped data into a dataframe
df_brueter = pd.DataFrame(list_brueter)

In [ ]:
df_brueter.columns = ['web_id', 'bezirk', 'plz', 'ort', 'strasse', 'anhang', 'erstbeobachtung', 'beschreibung', 'besonderes', 'mauersegler',
            'kontrolle', 'sperling', 'ersatz', 'schwalbe', 'wichtig', 'star', 'sanierung', 'fledermaus', 'verloren', 'andere', 'hash']

In [5]:
# pickle the dataframe, we just took 6+ hours scraping this, don't lose it
df_brueter.to_pickle("data/nabu_data2.pkl")

<hr>

Quick look over the dataframe.

In [6]:
df_brueter.tail()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
2247,2303,Prenzlauer Berg,10409,Berlin,Naugarder Str. 5,,23.05.2022,"2022-05: Zwei Schwalbennester, unbekannt ob in...",Fotos folgen,0,...,0,0,1,0,0,0,0,0,0,45aac97ddb624fea5907e4edd7e6af97756511ccc6864e...
2248,2304,Treptow-Köpenick,12459,Berlin,"Wattstr. 69-70, Edison-Grundschule",,24.05.2022,Schulsanierung\r\n2022-05: Am gesamten Schul- ...,Schulsanierung: Gesamtsanierung Hauptgebäude (...,0,...,0,0,0,0,0,0,0,0,0,137ac24cf4631a1d51d62fb82fa45ef741ecab412df47a...
2249,2305,"Steglitz-Zehlendorf, Dahlem",14195,Berlin,Königin-Luise-Str. 43 und 41,Fotos,02.06.2022,Königin-Luise-Str. 43/Ecke Archivstraße (Backh...,,0,...,1,0,0,0,0,0,0,0,0,9f0c6fec5345ca44b9683f5fd4a096ff1f65e85f868bd0...
2250,2306,Steglitz-Zehlendorf,12163,Berlin,Gutsmuthstr. 21,Fotos (04.06.2022),04.06.2022,"2022-06:Haussperlinge Reviergesang, Ein- und A...",,0,...,1,0,0,0,0,0,0,0,0,8ccc4c6283d861aa4969b30aec3b0730c0bde7e57ede42...
2251,2307,Steglitz-Zehlendorf (Lankwitz),12247,Berlin,Mühlenstr. 36/Malteser Str.,Fotos (2022),07.06.2022,Online-Formular-Meldung\r\n\r\n2022-06: Unterh...,könnte Gefahr von Verschließen der Spechtlöche...,0,...,0,0,0,0,1,0,0,0,0,26ea594ee9ea2931188e4dc2e5fff010c9d69b270ec637...


In [9]:
df_brueter.shape

(2252, 21)

In [12]:
df_brueter.head()

,web_id,bezirk,plz,ort,strasse,anhang,erstbeobachtung,beschreibung,besonderes,mauersegler,...,sperling,ersatz,schwalbe,wichtig,star,sanierung,fledermaus,verloren,andere,hash
0,1,Wilmersdorf,14197,Berlin,Eberbacher Str. 24,Skizze,10.07.2003,"2017: wie Vorjahre, jedoch neuer Standort link...","Sanierungsmaßnahme im Sommer bis Herbst 18, Fa...",1,...,1,0,0,0,0,1,0,0,0,352513ead3632be7e10e157f7f4412b0cbb16e030eadf2...
1,2,Wilmersdorf,14197,Berlin,Eberbacher Str. 18,,10.07.2003,"2012 wieder alle, dazu rechts von Haustür Mitt...",,1,...,1,0,0,0,0,0,0,0,0,a4b52b7da2e5560519d20194d5cbae277df57558e2e54f...
2,3,Wilmersdorf,14197,Berlin,Eberbacher Str. 30/Schlangenbader Str. 80,Skizzen und Fotos,10.07.2003,"2017: wie Vorjahre\r\n2014: 2x Mauersegler, di...","Sanierungsmaßnahme im Sommer bis Herbst 18, Fa...",1,...,1,0,0,1,0,0,0,0,0,199c11003859ef992f6718121436a47b10aaa6e54fcacd...
3,4,Wilmersdorf,14197,Berlin,Landauer Str. 2,,12.07.2003,Mauersegler und Sperlinge unter der Dachrinne ...,,1,...,1,0,0,0,0,0,0,0,0,44df52f1cd35a29a6d881fc8c7f987233355507a4af43e...
4,5,Wilmersdorf,14197,Berlin,Landauer Str. 9,,12.07.2003,2009: drei Mauerseglereinflüge und 2 Sperlings...,wichtiger Mauerseglerstandort,1,...,1,0,0,1,0,0,0,0,0,f19ca1f08a48a0d55021797808aa462ab869971126a509...


In [13]:
df_brueter.dtypes

web_id              int64
bezirk             object
plz                object
ort                object
strasse            object
anhang             object
erstbeobachtung    object
beschreibung       object
besonderes         object
mauersegler         int64
kontrolle           int64
sperling            int64
ersatz              int64
schwalbe            int64
wichtig             int64
star                int64
sanierung           int64
fledermaus          int64
verloren            int64
andere              int64
hash               object
dtype: object

In [14]:
testing = df_brueter.iloc[2]['beschreibung']
testing

'2017: wie Vorjahre\r\n2014: 2x Mauersegler, direkt an Ecke und ca 1-2 m von Ecke Front Schlagenbader, sowie Sperlinge verteilt unter der Dachrinne, auch am kleinen Dach über Balkonfront Schlangenbader\r\n2009: 1x Sperlinge an Ecke, 1x Sperlinge 50 cm von Ecke Front Schlangenbader, dann 02.06.09 3x Mauersegler, 1x an alter Stelle direkt Ecke, 1x ca. 1m in Front Schlangenbader, 1x Schlangenbader 80 Mitte rechter Balkon, alle unter DR -\r\nan der Hausecke unter der Dachrinne, in der Front, die zur Schlangenbader Str. zeigt, Eckhaus zur Schlangenbader Str. 80\r\nSperlinge und Mauersegler\r\nlinke Ecke Mauersegler (Richtung Schlangenbader), rechte Ecke und linke Ecke Sperlinge\r\n2005: auch\r\n2006: Front Schlangenbader Str. ca. 1m von Ecke Sperlinge, Front Eberbacher Str. ca. 50 cm und 2m von Ecke Sperlinge, linke Ecke Mauersegler (Richtung Schlangenbader Str.) 18.05.\r\n2007: Front Schlange 1m von Ecke Sperlinge, sowie erstes Fenster links von Haustür Sperlinge,1x Mauersegler Front zur S